---
Author details: EcoCommons team 2024? 
Editor details:     
Contact details: support@ecocommons.org.au 
Copyright statement: This script is the product of the EcoCommons platform.   
                     Please refer to the EcoCommons website for more details:   
                     <https://www.ecocommons.org.au/>  
Date: August 2024  

---

#### Script and data info: 

Use Case: Locating distribution of High density migratory shore birds to identify hotspots in QLD, which can assist to manage and contibute to Avian flu studies.

1. Bar-tailed Godwit (Limosa lapponica)
Density: Very high
Notes: One of the most common and numerous shorebirds in Australia during migration. Known for their long non-stop flights.
2. Red-necked Stint (Calidris ruficollis)
Density: Very high
Notes: This small shorebird is highly abundant in Australia, often seen in large flocks.
3. Curlew Sandpiper (Calidris ferruginea)
Density: High
Notes: Once common, but numbers have declined significantly in recent years due to habitat loss along the flyway.
4. Great Knot (Calidris tenuirostris)
Density: High
Notes: Another abundant species in Australia, although populations are under threat.
5. Sharp-tailed Sandpiper (Calidris acuminata)
Density: High
Notes: Commonly found in wetlands and marshes across Australia.

In [1]:
# Install and load necessary packages
install.packages("rgbif")
library(rgbif)
library(sf)

# Define the species and region of interest
species_list <- c("Limosa lapponica", "Calidris ruficollis", 
                  "Calidris ferruginea", "Calidris tenuirostris", 
                  "Calidris acuminata")

# Create an empty list to store data
all_occurrences <- list()

# Loop through each species and download data
for (species in species_list) {
  occ_data <- occ_search(scientificName = species, 
                         country = "AU", 
                         stateProvince = "Queensland", 
                         year = 2014:2024, 
                         limit = 1000)
  
  # Add species name to the data
  occ_data$data$species <- species
  
  # Convert to a spatial object
  coords <- occ_data$data[, c("decimalLongitude", "decimalLatitude")]
  bird_sf <- st_as_sf(occ_data$data, coords = coords, crs = 4326, agr = "constant")
  
  # Append to the list
  all_occurrences[[species]] <- bird_sf
}

# Combine all species data into one spatial object
combined_sf <- do.call(rbind, all_occurrences)

# Plot the points
plot(st_geometry(combined_sf), main = "Bird Observations in Queensland (2014-2024)")


also installing the dependencies 'triebeard', 'urltools', 'httpcode', 'plyr', 'crul', 'data.table', 'oai'





The downloaded binary packages are in
	/var/folders/t0/9dty7yc10v91d98_j_6hqpj00000gn/T//Rtmp4zCSBm/downloaded_packages


Linking to GEOS 3.11.0, GDAL 3.5.3, PROJ 9.1.0; sf_use_s2() is TRUE



ERROR: Error in occ_data$data[, c("decimalLongitude", "decimalLatitude")]: incorrect number of dimensions


In [ ]:
# Install and load necessary packages
install.packages("spatstat")
library(spatstat)

# Convert to a point pattern object for spatial analysis
bird_ppp <- as.ppp(combined_sf)

# Apply Kernel Density Estimation
bird_kde <- density(bird_ppp, sigma = 0.1)  # Adjust sigma for smoothing

# Plot the KDE
plot(bird_kde, main = "Kernel Density Estimation of Bird Observations in QLD")


In [ ]:
# Install and load necessary packages
install.packages("biomod2")
library(biomod2)

# Prepare data for SDM
myBiomodData <- BIOMOD_FormatingData(resp.var = as.numeric(combined_sf$species %in% species_list),
                                     expl.var = my_environmental_layers, 
                                     resp.xy = st_coordinates(combined_sf),
                                     resp.name = species_list)

# Run the model
myBiomodModelOut <- BIOMOD_Modeling(myBiomodData, models = c('GLM', 'RF'), 
                                    models.options = myBiomodOptions, 
                                    NbRunEval = 3, DataSplit = 80)

# Project the model
myBiomodProj <- BIOMOD_Projection(modeling.output = myBiomodModelOut, 
                                  new.env = my_new_environmental_layers)

# Plot the projected distribution
plot(myBiomodProj, main = "Projected Distribution of Selected Bird Species in QLD")
